# <font color = 'pickle'>**HW4 - 20 Points** </font>

1. For Task (1, 3 and 4) you will submit two files: a) A colab notebook b) A well formatted PDF file (convert notebook tp pdf).
2. The notebook and pdf files should contain all the output.
3. For Task 2 - submit a pdf OR ppt file.
4. Name the files as follows : FirstName_hw4.ipynb, FirstName_hw4.pdf
5. If the submission requires multiple files name them as follows: FirstName_file1_hw4, FirstName_file2_hw4.

# <font color = 'pickle'> **Install/Import Modules**

In [1]:
if 'google.colab' in str(get_ipython()):
    print('Running on Colab')
else:
    print('Not Running on Colab')

Running on Colab


In [2]:
if 'google.colab' in str(get_ipython()):
    !pip install  -U spacy  -q

In [3]:
import spacy 
import pandas as pd
from pathlib import Path
import sys
import re
import joblib

In [4]:
spacy.__version__

'3.4.1'

In [5]:
%load_ext autoreload
%autoreload 2

# <font color = 'pickle'> **Specify Project Folder Location**

In [6]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')     

Mounted at /content/drive


In [7]:
if 'google.colab' in str(get_ipython()):
    base_folder = Path('/content/drive/MyDrive/NLP_Fall22/HW4/')
else:
    base_folder = Path('/home/harpreet/Insync/google_drive_shaannoor/data/')

In [8]:
data_folder = base_folder/'Data/'
#archive_folder = base_folder/'archive'
custom_function_folder = base_folder/'Scripts/custom_functions'

In [9]:
sys.path.append(str(custom_function_folder))

In [10]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/NLP_Fall22/HW4/Scripts/custom_functions']

In [11]:
if 'google.colab' in str(get_ipython()):
    !python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 8.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# <font color = 'pickle'>  **Task 1 Rule Based Matching - 5 points**

## <font color = 'pickle'> **Get Data**

For this HW we wil use a subset of United Nations General Debate Corpus. 
    
Reference: Jankin Mikhaylov, Slava; Baturo, Alexander; Dasandi, Niheer, 2017, "United Nations General Debate Corpus", https://doi.org/10.7910/DVN/0TJX8Y, Harvard Dataverse, V6
    
**Data Description from [here](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/0TJX8Y)**
"*Every year since 1946, representatives of the UN member states gather at the annual sessions of the United Nations General Assembly. The centrepiece of each session is the General Debate. This is a forum at which leaders and other senior officials deliver statements that present their government’s perspective on the major issues in world politics.*"
    
You can download the individual text files form here <font color = 'indianred'> UNGDC_1970-2020.tar.gz </font> from this [link](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/0TJX8Y).
    
 I have combined the speeches in to a dataframe. I have also extracted year , session number and country as columns. Further, i have done basic cleaning. <font size = 4, color = indianred>**For the HW, please load the dataframe using joblib from file 'ungdc_cleaned_df.joblib'.**</font>

In [ ]:
# change the path to where you have downloaded the file - 'ungdc_cleaned_df.joblib'
df = joblib.load(data_folder/'ungdc_cleaned_df.joblib')

In [ ]:
df.head()

,Unnamed: 0,years,countries,sessions,speeches,cleaned_speeches
0,0,1970,ALB,25,33: May I first convey to our President the co...,33: May I first convey to our President the co...
1,1,1970,ARG,25,177.\t : It is a fortunate coincidence that pr...,: It is a fortunate coincidence that precisel...
2,2,1970,AUS,25,100.\t It is a pleasure for me to extend to y...,"It is a pleasure for me to extend to you, Mr..."
3,3,1970,AUT,25,155.\t May I begin by expressing to Ambassado...,May I begin by expressing to Ambassador Hamb...
4,4,1970,BEL,25,"176. No doubt each of us, before coming up to ...","176. No doubt each of us, before coming up to ..."


In [ ]:
df['countries'].value_counts()

ALB    51
IND    51
IRQ    51
ISL    51
ISR    51
       ..
MNE    15
SRB    15
EU     10
SSD     9
POR     1
Name: countries, Length: 201, dtype: int64

## <font color = 'pickle'> **Get Subset of Data**

For the HW we will use subset of data. We will extract data for USA and India. The Country codes are "USA" and "IND". 

In [ ]:
df_subset = df[(df['countries']=='USA') | (df['countries']=='IND')].copy()

In [ ]:
df_subset['countries'].value_counts()

IND    51
USA    51
Name: countries, dtype: int64

In [ ]:
df_subset.head()

,Unnamed: 0,years,countries,sessions,speeches,cleaned_speeches
26,26,1970,IND,25,"40.\t Mr. President, I offer you our congratul...","Mr. President, I offer you our congratulation..."
65,65,1970,USA,25,1.\t It is my privilege to extend to you once ...,It is my privilege to extend to you once agai...
114,114,1971,IND,26,"38.\tMr. President, on behalf of the people of...","Mr. President, on behalf of the people of Indi..."
179,179,1971,USA,26,"Mr. President, I should like first of all to ...","Mr. President, I should like first of all to ..."
238,238,1972,IND,27,"Mr. President, I offer you on behalf of India ...","Mr. President, I offer you on behalf of India ..."


## <font color = 'pickle'>  **Task Description**

For the task you will use the subset of data related to speeches of India and USA only. We have extracted this subset above.

In this task you will have to extract top ten (based on frequency) initiatives mentioned in speeches from USA and India. The initiatives are defined based on follwing set 

```
initiative_list = ['programme','scheme',
                 'initiative','campaign',
                 'agreement','conference',
                 'alliance','plan']
```

Some examples of initiatives will be - (1) United Nations International Drug Control Programme (2) Global Peace Operations Initiative (3) Global Alliance.

Negative exaples -- (1) good plan, (2) excellent initiative etc.

- The initiatives can be two or more tokens. The last token will be the token from the list above.
- You have to extract the unique initiatives from each speech (Hint : convert list of matches to set)
- Finally, you have to provide the top ten initiatives in speeches across the years for USA and India.


## <font color = 'pickle'>**Understanding Structure of initiatives/plans/schemes**

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
sample = '''The United States will continue its strong efforts to advance the United Nations plan for Namibia.The United States and Italy have proposed a Global Peace Operations Initiative
For 35 years, United Nations International Drug Control Programme has helped. This scheme was good. It was a great plan'''

In [ ]:
doc = nlp(sample)
print(f"{'token.text':<15} -> {'token.pos_':<10} -> {'token.dep_':<10}")
for token in doc:
    print(f'{token.text:<15} -> {token.pos_:<10} -> {token.dep_:<10}')

token.text      -> token.pos_ -> token.dep_
The             -> DET        -> det       
United          -> PROPN      -> compound  
States          -> PROPN      -> nsubj     
will            -> AUX        -> aux       
continue        -> VERB       -> ROOT      
its             -> PRON       -> poss      
strong          -> ADJ        -> amod      
efforts         -> NOUN       -> dobj      
to              -> PART       -> aux       
advance         -> VERB       -> acl       
the             -> DET        -> det       
United          -> PROPN      -> compound  
Nations         -> PROPN      -> compound  
plan            -> NOUN       -> dobj      
for             -> ADP        -> prep      
Namibia         -> PROPN      -> pobj      
.               -> PUNCT      -> punct     
The             -> DET        -> det       
United          -> PROPN      -> compound  
States          -> PROPN      -> nsubj     
and             -> CCONJ      -> cc        
Italy           -> PROPN      ->

## <font color = 'pickle'> **Get Initiatives**

In [ ]:
initiative_list = ['programme','scheme',
                 'initiative','campaign',
                 'agreement','conference',
                 'alliance','plan']

In [ ]:
from spacy.matcher import Matcher

In [ ]:
'''The United States will continue its strong efforts to advance the United Nations plan for Namibia.The United States and Italy have proposed a Global Peace Operations Initiative
For 35 years, United Nations International Drug Control Programme has helped. This scheme was good. It was a great plan'''

'The United States will continue its strong efforts to advance the United Nations plan for Namibia.The United States and Italy have proposed a Global Peace Operations Initiative\nFor 35 years, United Nations International Drug Control Programme has helped. This scheme was good. It was a great plan'

In [ ]:
matcher = Matcher(nlp.vocab)    
doc = nlp(sample)
pattern1 =  [{"pos": "PROPN"},
             {"pos": "PROPN", "OP": "*"},
             {'TEXT':{'IN': initiative_list+[a.capitalize() for a in initiative_list]}}]
matcher.add("mypattern",[pattern1],greedy='LONGEST')
matches = matcher(doc)
for match_id, start, end in matches:
  print(doc[start:end].text)

United Nations International Drug Control Programme
Global Peace Operations Initiative
United Nations plan


In [ ]:
def get_initiatives(text):
    nlp = spacy.load('en_core_web_sm')
    initiative_list = ['programme','scheme',
                 'initiative','campaign',
                 'agreement','conference',
                 'alliance','plan']
    
    pattern1 = [{"pos": "PROPN"},
             {"pos": "PROPN", "OP": "*"},
             {'TEXT':{'IN': initiative_list+[a.capitalize() for a in initiative_list]}}]
    matcher = Matcher(nlp.vocab)  
    matcher.add("mypattern",[pattern1],greedy='LONGEST')
    
    initiatives =[]
    for doc in nlp.pipe(text, batch_size=20, n_process=8):
      matches = matcher(doc)
      initiative_doc_base=[]
      for match_id, start, end in matches:      
        initiative_doc = doc[start:end].text
        initiative_doc_base.append(initiative_doc)
      initiative_doc_base = list(set(initiative_doc_base))
      initiatives.append(initiative_doc_base)
    return initiatives
    

In [ ]:
# if your pattern is correct, running this cell should give you the same result as result below
get_initiatives([sample])

[['United Nations plan',
  'United Nations International Drug Control Programme',
  'Global Peace Operations Initiative']]

In [ ]:
# DO NOT RUN THIS CELL

[['United Nations plan',
  'Global Peace Operations Initiative',
  'United Nations International Drug Control Programme']]

In [ ]:
# if your pattern is correct, running this cell should give you the same result as result below
get_initiatives([df_subset.loc[114,'cleaned_speeches']])

[['Fourth International Conference',
  'Quadripartite Agreement',
  'General Agreement']]

In [ ]:
# DO NOT RUN THIS CELL

[['General Agreement',
  'Quadripartite Agreement',
  'Fourth International Conference']]

In [ ]:
initiatives = get_initiatives(df_subset['cleaned_speeches'].values)

In [ ]:
df_subset['initiatives'] = initiatives

In [ ]:
# if your pattern is correct, running this cell should give you the same result as result below
df_subset.head()

,Unnamed: 0,years,countries,sessions,speeches,cleaned_speeches,initiatives
26,26,1970,IND,25,"40.\t Mr. President, I offer you our congratul...","Mr. President, I offer you our congratulation...",[United Nations Conference]
65,65,1970,USA,25,1.\t It is my privilege to extend to you once ...,It is my privilege to extend to you once agai...,"[Stockholm Conference, United Nations Conferen..."
114,114,1971,IND,26,"38.\tMr. President, on behalf of the people of...","Mr. President, on behalf of the people of Indi...","[Fourth International Conference, General Agre..."
179,179,1971,USA,26,"Mr. President, I should like first of all to ...","Mr. President, I should like first of all to ...","[Berlin agreement, Suez Canal agreement]"
238,238,1972,IND,27,"Mr. President, I offer you on behalf of India ...","Mr. President, I offer you on behalf of India ...","[Simla Agreement, Stockholm Conference, United..."


In [ ]:
# DO NOT RUN THIS CELL

,Unnamed: 0,years,countries,sessions,speeches,cleaned_speeches,initiatives
26,26,1970,IND,25,"40.\t Mr. President, I offer you our congratul...","Mr. President, I offer you our congratulation...",[United Nations Conference]
65,65,1970,USA,25,1.\t It is my privilege to extend to you once ...,It is my privilege to extend to you once agai...,"[United Nations Conference, United States init..."
114,114,1971,IND,26,"38.\tMr. President, on behalf of the people of...","Mr. President, on behalf of the people of Indi...","[General Agreement, Quadripartite Agreement, F..."
179,179,1971,USA,26,"Mr. President, I should like first of all to ...","Mr. President, I should like first of all to ...","[Suez Canal agreement, Berlin agreement]"
238,238,1972,IND,27,"Mr. President, I offer you on behalf of India ...","Mr. President, I offer you on behalf of India ...","[Simla Agreement, United Nations Conference, S..."


# <font color = 'pickle'> **Compare Initiatves of USA and India**

In [ ]:
from collections import Counter

In [ ]:
country = 'USA'



['Stockholm Conference',
 'United Nations Conference',
 'United States initiative',
 'Berlin agreement',
 'Suez Canal agreement',
 'United Nations Conference',
 'Suez Canal agreement',
 'Geneve Conference',
 'World Population Conference',
 'Third United Nations Conference',
 'World Food Conference',
 'Sinai agreement',
 'Third United Nations Conference',
 'Armistice Agreement',
 'Geneva Conference',
 'Armistice Agreement',
 'Geneva Conference',
 'Geneva Peace Conference',
 'Geneva Conference',
 'Geneva Peace Conference',
 'United Nations conference',
 'Bonn Economic Summit Conference',
 'World Food Conference',
 'United Nations Conference',
 'United Nations Conference',
 'Second Review Conference',
 'Review Conference',
 'United Nations plan',
 'Caribbean Basin initiative',
 'United Nations Conference',
 'Caribbean Basin Initiative',
 'Marshall Plan',
 'North Atlantic alliance',
 'General Agreement',
 'San Francisco Conference',
 'Caribbean Basin initiative',
 'Third Review Conference'

In [ ]:
# Write a function that takes country as an input and gives the counter object for that 
# country as output. 
# Hint:Start with an empty list. You will have to iterate over the initiative column 
# for the specific country # and extract all the initiatives in one list 
# (see append vs extend for python list). You will then pass the list to Counter Object.
def get_initiative_country(country: str):
    # code here
    all_initiatives = [item for sublist in list(df_subset[df_subset['countries']==country]['initiatives']) for item in sublist]
    return Counter(all_initiatives)   

In [ ]:
initiatives_usa = get_initiative_country('USA')

In [ ]:
initiatives_india = get_initiative_country('IND')

In [ ]:
# If your function is correct, running this cell should give you the similar result as result below
# The results need not be exactly the same.
initiatives_usa.most_common(10)

[('United Nations Conference', 6),
 ('General Agreement', 5),
 ('San Francisco Conference', 4),
 ('North American Free Trade Agreement', 4),
 ('Geneva Conference', 3),
 ('Marshall Plan', 3),
 ('Emergency Plan', 3),
 ('Stockholm Conference', 2),
 ('Suez Canal agreement', 2),
 ('Third United Nations Conference', 2)]

In [ ]:
# DO NOT RUN THIS CELL

[('United Nations Conference', 6),
 ('General Agreement', 5),
 ('San Francisco Conference', 4),
 ('North American Free Trade Agreement', 4),
 ('Geneva Conference', 3),
 ('Marshall Plan', 3),
 ('Emergency Plan', 3),
 ('Stockholm Conference', 2),
 ('Suez Canal agreement', 2),
 ('World Food Conference', 2)]

In [ ]:
# If your function is correct, running this cell should give you the similar result as result below
# The results need not be exactly the same.
initiatives_india.most_common(10)

[('United Nations Conference', 18),
 ('Simla Agreement', 8),
 ('International Conference', 7),
 ('United Nations plan', 5),
 ('Paris Agreement', 4),
 ('Third United Nations Conference', 4),
 ('International Solar Alliance', 4),
 ('General Agreement', 3),
 ('Seventh Conference', 3),
 ('United Nations Environment Programme', 2)]

In [ ]:
# DO NOT RUN THIS CELL

[('United Nations Conference', 18),
 ('Simla Agreement', 8),
 ('International Conference', 7),
 ('United Nations plan', 5),
 ('Paris Agreement', 4),
 ('Third United Nations Conference', 4),
 ('International Solar Alliance', 4),
 ('General Agreement', 3),
 ('Seventh Conference', 3),
 ('United Nations Environment Programme', 2)]

# <font color = 'pickle'>**Task 2 : Manual Backpropogation (5 Points)**

For the network below update the weights using back propogation. 

<img src = "https://drive.google.com/uc?export=view&id=1e1EI-N773mUtQJ0SQx5wlJJ1RaIQ4dZ2" width =600 >

- X1, x2 are inputs. The values of the inputs are provided.
- h1, h2 are hidden neurons. You will need to calculate the values of h1 and h2 in forward pass.
- o1 and o2 are outputs. 0.01 and 0.99 are the true values of the output. You will calculate the predicted values of o1 and o1 in forward pass.
- W1-W8 are weights. The initial values are provided to you. You will need to calculate the updated values in backward pass.
- bh1, bh2, bo1, bo2 are bias terms. The initial values are provided. You will need to calculate the updated values in backward pass.

- You will apply sigmoid activation on hidden layer.
- You will apply Linear activation function on output neurons.
- You will use the  squared error as the loss function. 
where <br>$ E_1 =1/2 *(\hat{o_1}-o_1)^2 $ <br> 
$ E_2 =1/2 *(\hat{o_2}-o_2)^2 $ <br>  $ E = E_1 + E_2$

Here $E$ is the total loss. $\hat{o_2}$ and $\hat{o_2}$ are predicted values of $o_1$ and $o_2$.
 
- <font color ='indianred'> **Assume a Learning Rate of 10.**
    

<font size = 4 color ='indianred'> **Requirements**
- Show caluclations for one forward and one backward pass.
- Show all the steps of your calculations. You will get partial credit for the steps even if the final answers are not accurate.
- You will do this question manually.
- For this question you can submit - ppt  or pdf file (pdf of handwritten calculations).

# <font color = 'pickle'>**Task 3 - Tensors and Autodiff - 5 Points**

In [ ]:
import torch

## <font color = 'pickle'>**Q1 : Create Tensor (1/2 Point)**
 Create a torch Tensor of shape (5, 3) which is filled with zeros. Modify the tensor to set element (0, 2) to 10 and element (2, 0)  to 100.

In [ ]:
tens = torch.zeros(5,3)      
tens[0,2] = 10
tens[2,0] = 100
print(tens)

tensor([[  0.,   0.,  10.],
        [  0.,   0.,   0.],
        [100.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.]])


In [ ]:
# DO NOT RUN THIS CELL

tensor([[  0.,   0.,  10.],
        [  0.,   0.,   0.],
        [100.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.]])

## <font color = 'pickle'>**Q2: Reshape tensor (1/2 Point)**
You have following tensor as input:

```x=torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])```

Using only reshaping functions (like view, reshape, transpose, permute), you need to get at the following tensor as output:

```
tensor([[ 0,  4,  8, 12, 16, 20],
        [ 1,  5,  9, 13, 17, 21],
        [ 2,  6, 10, 14, 18, 22],
        [ 3,  7, 11, 15, 19, 23]])
```



In [ ]:
x = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])
x = torch.reshape(x,(6,4))
x = torch.transpose(x,0,1)
x

tensor([[ 0,  4,  8, 12, 16, 20],
        [ 1,  5,  9, 13, 17, 21],
        [ 2,  6, 10, 14, 18, 22],
        [ 3,  7, 11, 15, 19, 23]])

In [ ]:
# DO NOT RUN THIS CELL

tensor([[ 0,  4,  8, 12, 16, 20],
        [ 1,  5,  9, 13, 17, 21],
        [ 2,  6, 10, 14, 18, 22],
        [ 3,  7, 11, 15, 19, 23]])

## <font color = 'pickle'>**Q3: Slice tensor (1 Point)**

- Slice the tensor x to get the following
>- last row of x
>- fourth column of x
>- first three rows and first two columns - the shape of subtensor should be (3,2)
>- odd valued rows and columns

In [ ]:
x = torch.tensor([[1, 2, 3, 4, 5], [6, 7, 8, 8, 10], [11, 12, 13, 14, 15]])
x

tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  8, 10],
        [11, 12, 13, 14, 15]])

In [ ]:
x.shape

torch.Size([3, 5])

In [ ]:
# get last row fo the tensor
last_row = x[len(x)-1]
last_row

tensor([11, 12, 13, 14, 15])

In [ ]:
fourth_column = x[:,3]
fourth_column

tensor([ 4,  8, 14])

In [ ]:
first_3_rows_2_columns = x[0:3,0:2]
first_3_rows_2_columns

tensor([[ 1,  2],
        [ 6,  7],
        [11, 12]])

In [ ]:
odd_valued_rows_columns = x[0:3:2,0:5:2]
odd_valued_rows_columns

tensor([[ 1,  3,  5],
        [11, 13, 15]])

##  <font color = 'pickle'>**Q4 -Normalize Function (1 Points)**<font>

Write the function that normalizes the columns of a matrix. You have to compute the mean and standard deviation of each column. Then for each element of the column, you subtract the mean and divide by the standard deviation.

In [ ]:
x = [[ 3,  60,  100, -100],
     [ 2,  20,  600, -600],
     [-5,  50,  900, -900]]

In [ ]:
X = torch.tensor(x)
X= X.float()

In [ ]:
print(X.shape)
print(X.dtype)

torch.Size([3, 4])
torch.float32


In [ ]:
X.mean(axis = 0)

tensor([   0.0000,   43.3333,  533.3333, -533.3333])

In [ ]:
X.std(axis = 0)

tensor([  4.3589,  20.8167, 404.1452, 404.1452])

In [ ]:
def noramlize_matrix(x):
  mean = X.mean(axis = 0)
  std = X.std(axis = 0)
  y = (x-mean)/std
  return y
  

In [ ]:
# if your function is correct, running this cell should give you the same result as result below
Z = noramlize_matrix(X)
Z

tensor([[ 0.6882,  0.8006, -1.0722,  1.0722],
        [ 0.4588, -1.1209,  0.1650, -0.1650],
        [-1.1471,  0.3203,  0.9073, -0.9073]])

In [ ]:
# DO NOT RUN THIS CELL

tensor([[ 0.6882,  0.8006, -1.0722,  1.0722],
        [ 0.4588, -1.1209,  0.1650, -0.1650],
        [-1.1471,  0.3203,  0.9073, -0.9073]])

In [ ]:
# if your function is correct, running this cell should give you the same result as result below
Z.mean(axis = 0)

tensor([ 0.0000e+00,  4.9671e-08,  3.9736e-08, -3.9736e-08])

In [ ]:
# DO NOT RUN THIS CELL

tensor([ 0.0000e+00,  4.9671e-08,  3.9736e-08, -3.9736e-08])

In [ ]:
# if your function is correct, running this cell should give you the same result as result below
Z.std(axis = 0)

tensor([1., 1., 1., 1.])

In [ ]:
# DO NOT RUN THIS CELL

tensor([1., 1., 1., 1.])

##  <font color = 'pickle'>**Q5: Matrix Multiplication on CPU vs GPU- 1 Point**<font>

### <font color = 'pickle'>**Q5 Part A CPU**<font>

- Compute the matrix multiplication of matrix x1 and y1 below. The final matrix should have the shape (10000, 10000). You might  have to reshape one of the matrix to do this operation. 



In [ ]:
import time

x1 = torch.rand(5000, 10000)
y1 = torch.rand(5000, 10000)
x1 = x1.reshape(10000,5000)

t0 = time.time()

# matrix multiplication of x1, y1
z1 = torch.mm(x1, y1)
t1 = time.time()

print('shape of z1:', z1.shape == (10000, 10000))
print('z on CPU:', z1.device == torch.device('cpu'))

cpu_time = 1000.0 * (t1 - t0)
print(f'CPU time: {cpu_time:.2f}')


shape of z1: True
z on CPU: True
CPU time: 13004.88


In [ ]:
# check gpu is availible - if not change the runtime type 
# Go to Runtime -- change run time type
if torch.cuda.is_available():
  print('GPU is availible')
else:
  print('GPU is not availible')

GPU is availible


###  <font color = 'pickle'>**Q5 Part B GPU**<font>
(b) Now you wil do the  matrix multiplication of matrix x2 and y2. The final matrix should have the shape (10000, 10000). However before doing the matrix multiplication, move x2 and y2 on gpus. After doing the multiplication move z2 back to cpu. 

In [ ]:
x2 = torch.rand(5000, 10000)
y2 = torch.rand(5000, 10000)

t0 = time.time()
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
# move x to gpu
x2 = x2.to(device = device)

# move y to gpu
y2 = x2.to(device = device)

# matrix multiplication of x2, y2
x2 = x2.reshape(10000, 5000)
z2 =  torch.mm(x2, y2)

# waits for everything to finish running
torch.cuda.synchronize()
t1 = time.time()

# move z2 to cpu
z2 = z2.to('cpu')

print('shape of z2:', z2.shape == (10000, 10000)) # This should print True
print('z2 on CPU:', z2.device == torch.device('cpu')) # This should print cpu

gpu_time = 1000.0 * (t1 - t0)
print(f'GPU time: {gpu_time:.2f}')

shape of z2: True
z2 on CPU: True
GPU time: 6182.09


##  <font color = 'pickle'>**Q6 -Calculate Gradients 1 Point** 

Compute Gradient using  PyTorch Autograd - 2 Points
## $f(x,y) = \frac{x + \exp(y)}{\log(x) + (x-y)^3}$
Compute dx and dy at x=3 and y=4

In [65]:
import math
import numpy as np
from numpy import exp
def fxy(x,y):
  num = x + torch.exp(y)
  den = torch.log(x) + torch.pow(x-y,3)
  return num/den

In [66]:
x = torch.tensor([3.0],requires_grad=True) 
y = torch.tensor([4.0],requires_grad=True) 
# I changed the code for y here as I wanted to compute the gradient of y at the end of this task. requires_grad as False, gives me None value for dy
print(x)

tensor([3.], requires_grad=True)


In [67]:
# if your function is correct, running this cell should give you the same result as result below
f = fxy(x,y)
f

tensor([584.0868], grad_fn=<DivBackward0>)

In [68]:
# DO NOT RUN THIS CELL

In [69]:
# compute gradients
f.backward()
print('gradients using pytorch autodiff function \n')
print(f'gradient with respect to a  : {x.grad}')
print(f'gradient with respect to a  : {y.grad}')

gradients using pytorch autodiff function 

gradient with respect to a  : tensor([-19733.3965])
gradient with respect to a  : tensor([18322.8477])


In [70]:
print(x.grad, y.grad)

tensor([-19733.3965]) tensor([18322.8477])


# <font color = 'pickle'> **Task 4: Logistic Regression using Minibatch Stochastic Gradient Descent with PyTorch - 5 Points**

## <font color = 'pickle'>**Data**

In [90]:
from sklearn.datasets import make_moons
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [91]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [92]:
X, y = make_moons(n_samples=1000, noise=0.05, random_state=0)

In [93]:
preprocessor = StandardScaler()
X = preprocessor.fit_transform(X)

In [94]:
print(X.shape, y.shape)

(1000, 2) (1000,)


In [95]:
X[0:5]

array([[ 1.75081891,  0.48393978],
       [ 1.35606443, -0.90706929],
       [-0.90150442,  1.22470664],
       [-0.60117222, -0.14688373],
       [ 0.0048725 , -1.28700543]])

In [96]:
print(y[0:10])

[1 1 0 1 1 1 0 0 0 1]


## <font color = 'pickle'>**Dataset and Data Loaders**

In [97]:
# Initializing the batch size
batch_size = 2

# Create Tensors from numpy 
x_tensor = torch.from_numpy(X).float()
y_tensor = torch.from_numpy(y.reshape(-1, 1)).float()
y_tensor = torch.squeeze(y_tensor)
# create a PyTorch Dataset from x and y tensors
train_dataset = TensorDataset(x_tensor,y_tensor)

# create Data loader from Dataset
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, 
                                            batch_size = batch_size, 
                                            shuffle = True)

## <font color = 'pickle'>**Model**

In [98]:
# Speciy your model
def softmax_reg(n_ins, n_outs, bias):
  return nn.Sequential(nn.Linear(n_ins, n_outs))

num_inputs = train_dataset.tensors[0].shape[1]
num_outputs = 2
model = softmax_reg(num_inputs, num_outputs, bias = True)


## <font color = 'pickle'>**Loss function**

In [99]:
# specify your loss function
loss_function =  nn.CrossEntropyLoss()

## <font color = 'pickle'>**Optimizer**

In [100]:
# specify optimizer
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## <font color = 'pickle'>**Initialization**

Create a function to initilaize weights. 
- Initialize weights using normal distribution with mean = 0 and std = 0.05 
- Initilaize the bias term with zeros

In [101]:
def init_weights(layer):
  if type(layer) == nn.Linear:
    torch.nn.init.normal_(layer.weight, mean = 0, std = 0.05)
    torch.nn.init.zeros_(layer.bias)

## <font color = 'pickle'>**Training Loop**

**Model Training** involves five steps: 

- Step 0: Randomly initialize parameters / weights
- Step 1: Compute model's predictions - forward pass
- Step 2: Compute loss
- Step 3: Compute the gradients
- Step 4: Update the parameters
- Step 5: Repeat steps 1 - 4

Model training is repeating this process over and over, for many **epochs**.

We will specify number of ***epochs*** and during each epoch we will iterate over the complete dataset and will keep on updating the parameters.

***Learning rate*** and ***epochs*** are known as hyperparameters. We have to adjust the values of these two based on validation dataset.

We will now create functions for step 1 to 4.

In [102]:
epochs = 5
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# move model tp gpu
model.to(device)

# Step 0: Randomly initialize parameters / weights

model.apply(init_weights)

for epoch in range(epochs):

  # Training Loop 
  # Initialize train_loss at the he start of the epoch
  running_train_loss = 0
  running_train_correct = 0
  correct = torch.Tensor() # empty tensor
  correct = correct.to(device)

  # Iterate on batches from the dataset using train_loader
  for input_, targets in train_loader:
    
    # move inputs and outputs to GPUs
    input_ = input_.to(device)
    targets = targets.type(torch.LongTensor)
    targets = targets.to(device)

    # Step 1: Forward Pass: Compute model's predictions 
    output = model(input_)
    
    # Step 2: Compute loss
    loss = loss_function(output, targets)

    # Correct prediction
    # get the predicted class
    y_pred = torch.argmax(output, dim = 1)

    # get total correct predictions for the batch
    correct = torch.sum(y_pred == targets)

    # Step 3: Backward pass -Compute the gradients

    # set gradients to zero
    optimizer.zero_grad()
    
    # calculate gradients
    loss.backward()

    # Step 4: Update the parameters
    optimizer.step()
          
    # Add train loss of a batch 
    running_train_loss += loss.item()

    # Add Corect counts of a batch
    running_train_correct += correct
  
  # Calculate mean train loss for the whole dataset for a particular epoch
  train_loss = running_train_loss/len(train_loader)

  # Calculate accuracy for the whole dataset for a particular epoch
  train_acc = running_train_correct/len(train_loader.dataset)

  # Print the train loss and accuracy for given number of epochs, batch size and number of samples
  print(f'Epoch : {epoch+1} / {epochs}')
  print(f'Train Loss: {train_loss : .4f} | Train Accuracy: {train_acc * 100 : .4f}%')

cuda:0
Epoch : 1 / 5
Train Loss:  0.2636 | Train Accuracy:  87.7000%
Epoch : 2 / 5
Train Loss:  0.2444 | Train Accuracy:  88.5000%
Epoch : 3 / 5
Train Loss:  0.2457 | Train Accuracy:  87.7000%
Epoch : 4 / 5
Train Loss:  0.2444 | Train Accuracy:  88.4000%
Epoch : 5 / 5
Train Loss:  0.2463 | Train Accuracy:  88.5000%


In [103]:
# print the estimated weight and bias term
for name, param in model.named_parameters():
  print(name, param.data)

0.weight tensor([[-0.5163,  1.5884],
        [ 0.4170, -1.4683]], device='cuda:0')
0.bias tensor([ 0.0250, -0.0250], device='cuda:0')
